In [1]:
# TODO check lit function
# TODO check agg function
# TODO flightsDF.sample(False, 0.1).take(2)
# TODO countDistinct()
# TODO from pyspark.sql.functions import count, round
# TODO When otherwise
# TODO from pyspark.sql.functions import max, min, avg, stddev
# TODO cast, join, limit, pivot, withColumnRenamed
# TODO Create manual DataFrame
# TODO .toPandas()

# TODO write introduction with goals set and what could be improved in the future
        # not only copy paste from stack overflow, profssor, but understand the code, play around with function-parameters from professor
        # use code from the book "Spark - The Definitive Guide" by Bill Chambers and Matei Zaharia -> i have bought it because I am such a Nerd
        # have a well commented/documented & modular code 
        # create some useful functions and automate as much as possible
        # use all of the following capabilities:
        # flightsDF.sample(False, 0.1).take(2)
        #  

# Spark Individual Assignment - Maximilian Pichler - MBD2020

As part of our Spark Class, we have learned how to use the pyhton API "pyspark" to perform dataprocessing with spark.

This is the notebook used for the analysis and processing of the hotel dataset from [kaggle.com](https://www.kaggle.com/jessemostipak/hotel-booking-demand)


## Content
### Import libraries
    - pyspark
    - KaggleAPI
    
### Import data
    - via CSV
    - via Kaggle API

### Perform basic EDA
    - infer schema from dataset
    - get shape of dataset
    - identify entities, metrics and dimesions
    - categorize columns

### Perform basic profiling based on previous grouping
    - count and display unique values
    - create summary statistics of numeric columns
### Answer Business Questions:
    - Question 1: what are our most profitable customers
    - Question 2: at what time do we get the most profitable bookings
    - Question 3: what are features of bookings that take long (waitinglist)

In [2]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import when, count, col, countDistinct, desc, asc, first, lit, round, date_format, concat_ws, to_date, date_add, expr, month
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType, Row
from pathlib import Path
from IPython.display import display, Markdown

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


# Import Data


In [3]:
data_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv('Data/Spark Lab/Individual Assignment/hotel_bookings.csv')
#data_df.cache()

In [4]:
# assigning the schema and column names to variables
schema = data_df.schema
columns = data_df.schema.names
total_bookings = data_df.count()


print('printing the schema of the dataset')
data_df.printSchema()

print(f'the dataset consists of {total_bookings} rows')

# TODO Typecasts
# TODO change some columnnames
# TODO delete some column

print('printing null values per column')
# thank you Raúl for this line of code!
# it took a while to understand it, it is genius!
data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in columns[:10]]).show()             
data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in columns[10:19]]).show()
data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in columns[19:29]]).show()
data_df.select([count(when(col(c).isNull(), c)).alias(c) for c in columns[29:]]).show()


printing the schema of the dataset
root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |--

In this data set, consisting of 119.390 rows we can see a mix of integer-, double- and string-type data. The initial check for NULL values (with .isNull()) suggested that there are no missing values. However, running the code again with == 'NULL' we can see that the columns "company", "agent", and "country" have some missing values.

In [5]:
print('printing null values per column')
# thank you Raúl for this line of code!
# it took a while to understand it, it is genius!
data_df.select([count(when(col(c).isNull() | (col(c) == "NULL"), c)).alias(c) for c in columns[:10]]).show()             
data_df.select([count(when(col(c).isNull() | (col(c) == "NULL"), c)).alias(c) for c in columns[10:19]]).show()
data_df.select([count(when(col(c).isNull() | (col(c) == "NULL"), c)).alias(c) for c in columns[19:29]]).show()
data_df.select([count(when(col(c).isNull() | (col(c) == "NULL"), c)).alias(c) for c in columns[29:]]).show()

printing null values per column
+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|
+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+
|    0|          0|        0|                0|                 0|                       0|                        0|                      0|                   0|     0|
+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+

+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+----------

Next, lets try to define some groups of columns. For this reason, we will first check [the datasource](https://www.kaggle.com/jessemostipak/hotel-booking-demand) and get ourselves familiar with the contents of each column:

- 'hotel' - Category telling us whether it is a Resort Hotel or a City Hotel.
- 'is_canceled' - indication whether the booking was cancelled
- 'lead_time' - time between booking and the arrival date
- 'arrival_date_year' - Year of arrival
- 'arrival_date_month' - month of arrival
- 'arrival_date_week_number' - day of week of arrival
- 'arrival_date_day_of_month' - day of month of arrival
- 'stays_in_weekend_nights' - number of weekend nights of the stay
- 'stays_in_week_nights' - number of nights of the stay
- 'adults' - number of adults
- 'children' - number of children
- 'babies' - number of babies
- 'meal' - type of meal booked (BB, HB, FB, SC/Undefined)
- 'country' - Country of origin
- 'market_segment' - Market Segments (TA = Travel Agent)
- 'distribution_channel' - booking distribution channel
- 'is_repeated_guest' - whether the guest had previous bookings
- 'previous_cancellations' - whether the guest had previous chancellations
- 'previous_bookings_not_canceled' - whether the guest had previous bookings that where not cancelled
- 'reserved_room_type' - reserved room type
- 'assigned_room_type' - assigned room type
- 'booking_changes' - number of changes to the booking
- 'deposit_type' - deposit type
- 'agent' - travel agency ID
- 'company' - ID of the entity that made/pays the booking
- 'days_in_waiting_list' - number of days it took to confirm the booking
- 'customer_type' - customer type
- 'adr' - Average Daily Rate
- 'required_car_parking_spaces' - number of parking spaces needed
- 'total_of_special_requests' - number of special requests
- 'reservation_status' - last reservation status
- 'reservation_status_date' - date at which the las reservation status was set


With this information at hand, we can further categorize the data, helping us to better understand what the data can tell us.

# Entries, Metrics & Dimensions

## Entities: 
- Bookings, 
- Guests (dimension)
- Hotels (dimension)
- Distribution Channels (dimension)

## Metrics: 
- average_daily_rate
- cancellations
- distribution channels
- ...

## Dimensions: 
- guests 
- agents 
- timing
- ...

# Column Categorization
- Booking related
- Timing related
- Guest/Targetgroup related

In [6]:
# define group variable "booking"
booking = ['is_canceled', 'market_segment','deposit_type', 'booking_changes', 
            'agent', 'days_in_waiting_list', 'reservation_status', 'hotel', 
            'assigned_room_type', 'distribution_channel', 'adr']

# define group variable "time"
time = ['lead_time', 'arrival_date_year', 'arrival_date_month', 
        'arrival_date_week_number', 'arrival_date_day_of_month', 
        'stays_in_weekend_nights', 'stays_in_week_nights', 'reservation_status_date']

# define group variable "guests"
guest = ['adults', 'children', 'babies', 'country', 'is_repeated_guest', 
        'previous_cancellations', 'company', 'required_car_parking_spaces', 
        'meal', 'reserved_room_type'] 

With these groups, we can now create some basic insights about the data.
We will query for dinsinct values, counts and summary statistics of numerical and categorical columns.

In [7]:
# Since we are going to use different metrics depending on the datatype, functions to get these appropriate columnnames can be handy.
# The basic idea here was to spend more time with one column-group, think about how to modularize functions so they can be used in multiple ways, to create a recipe that can be used regardless of the data at hand. This recipe can then be applied to the other column-groups.

def get_categoricals(data_df):
    """This function takes as input a spark dataframe and returns a list of its StringType columnames"""
    categoricals = [column.name for column in data_df.schema.fields if isinstance(column.dataType, StringType)]
    return categoricals


def get_numericals(data_df):
    """This function takes as input a spark dataframe and returns a list of its IntegerType and DoubleType columnames"""
    numericals =  [column.name for column in data_df.schema.fields if isinstance(column.dataType, (IntegerType, DoubleType))]
    return numericals

def get_min_max(data_df):
    for category in get_categoricals(data_df):
        first = data_df.groupBy(category).count().sort(desc('count')).first()
        last = data_df.groupBy(category).count().sort(asc('count')).first()
        display(Markdown("""
| %s | %s |
|----|----|
| %s | %s |
""" % (f"least_{category}", f"most_{category}",
    "%s (%d occurrences)" % (first[category], first["count"]), 
    "%s (%d occurrences)" % (last[category], last["count"]))))



# Basic profiling of booking-related data


In [8]:
print('\n print the most occuring entries of the whole booking group in descending order')
data_df.groupBy(booking).count().sort(desc('count')).show(10)

print('\n print the highest and lowest counts of categorical columns belonging to the booking-related group')
get_min_max(data_df[booking])

print('\n show number of unique categorical-values per column')
data_df.select([countDistinct(c).alias(c) for c in get_categoricals(data_df[booking])]).show()

print('\n print summary statistics of booking-numricals')
data_df.select(get_numericals(data_df[booking])).summary().show()



 print the most occuring entries of the whole booking group in descending order
+-----------+--------------+------------+---------------+-----+--------------------+------------------+------------+------------------+--------------------+-----+-----+
|is_canceled|market_segment|deposit_type|booking_changes|agent|days_in_waiting_list|reservation_status|       hotel|assigned_room_type|distribution_channel|  adr|count|
+-----------+--------------+------------+---------------+-----+--------------------+------------------+------------+------------------+--------------------+-----+-----+
|          1|        Groups|  Non Refund|              0|    1|                   0|          Canceled|  City Hotel|                 A|               TA/TO| 62.0| 2311|
|          1|        Groups|  No Deposit|              0|    1|                   0|          Canceled|  City Hotel|                 A|               TA/TO| 62.0|  662|
|          0|     Corporate|  No Deposit|              0| NULL|           


| least_market_segment | most_market_segment |
|----|----|
| Online TA (56477 occurrences) | Undefined (2 occurrences) |



| least_deposit_type | most_deposit_type |
|----|----|
| No Deposit (104641 occurrences) | Refundable (162 occurrences) |



| least_agent | most_agent |
|----|----|
| 9 (31961 occurrences) | 451 (1 occurrences) |



| least_reservation_status | most_reservation_status |
|----|----|
| Check-Out (75166 occurrences) | No-Show (1207 occurrences) |



| least_hotel | most_hotel |
|----|----|
| City Hotel (79330 occurrences) | Resort Hotel (40060 occurrences) |



| least_assigned_room_type | most_assigned_room_type |
|----|----|
| A (74053 occurrences) | L (1 occurrences) |



| least_distribution_channel | most_distribution_channel |
|----|----|
| TA/TO (97870 occurrences) | Undefined (5 occurrences) |



 show number of unique categorical-values per column
+--------------+------------+-----+------------------+-----+------------------+--------------------+
|market_segment|deposit_type|agent|reservation_status|hotel|assigned_room_type|distribution_channel|
+--------------+------------+-----+------------------+-----+------------------+--------------------+
|             8|           3|  334|                 3|    2|                12|                   5|
+--------------+------------+-----+------------------+-----+------------------+--------------------+


 print summary statistics of booking-numricals
+-------+-------------------+-------------------+--------------------+------------------+
|summary|        is_canceled|    booking_changes|days_in_waiting_list|               adr|
+-------+-------------------+-------------------+--------------------+------------------+
|  count|             119390|             119390|              119390|            119390|
|   mean|0.37041628277075134|0.2

# Basic profiling of timing-related data

In [9]:
print('\n print the most occuring entries of the whole timing group in descending order')
data_df.groupBy(time).count().sort(desc('count')).show(10)

print('\n print the highest and lowest counts of categorical columns belonging to the timing-related group')
get_min_max(data_df[time])

print('\n show number of unique categorical-values per column')
data_df.select([countDistinct(c).alias(c) for c in get_categoricals(data_df[time])]).show()

print('\n print summary statistics of time-numricals')
data_df.select(get_numericals(data_df[time])).summary().show()



 print the most occuring entries of the whole timing group in descending order
+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+-----------------------+-----+
|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|reservation_status_date|count|
+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+-----------------------+-----+
|      277|             2016|          November|                      46|                        7|                      1|                   2|             2016-04-04|  180|
|       68|             2016|          February|                       8|                       17|                      0|                   2|             2016-01-06|  150|
|       37|             2016|           Octob


| least_arrival_date_month | most_arrival_date_month |
|----|----|
| August (13877 occurrences) | January (5929 occurrences) |



| least_reservation_status_date | most_reservation_status_date |
|----|----|
| 2015-10-21 (1461 occurrences) | 2015-02-09 (1 occurrences) |



 show number of unique categorical-values per column
+------------------+-----------------------+
|arrival_date_month|reservation_status_date|
+------------------+-----------------------+
|                12|                    926|
+------------------+-----------------------+


 print summary statistics of time-numricals
+-------+------------------+------------------+------------------------+-------------------------+-----------------------+--------------------+
|summary|         lead_time| arrival_date_year|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|
+-------+------------------+------------------+------------------------+-------------------------+-----------------------+--------------------+
|  count|            119390|            119390|                  119390|                   119390|                 119390|              119390|
|   mean|104.01141636652986| 2016.156554150264|       27.16517296255968|       15.798241058715135| 

# Basic profiling of guest-related data

In [10]:
print('\n print the most occuring entries of the whole guest group in descending order')
data_df.groupBy(guest).count().sort(desc('count')).show(10)

print('\n print the highest and lowest counts of categorical columns belonging to the guest-related group')
get_min_max(data_df[guest])

print('\n show number of unique categorical-values per column')
data_df.select([countDistinct(c).alias(c) for c in get_categoricals(data_df[guest])]).show()

print('\n print summary statistics of guest-numricals')
data_df.select(get_numericals(data_df[guest])).summary().show()


 print the most occuring entries of the whole guest group in descending order
+------+--------+------+-------+-----------------+----------------------+-------+---------------------------+----+------------------+-----+
|adults|children|babies|country|is_repeated_guest|previous_cancellations|company|required_car_parking_spaces|meal|reserved_room_type|count|
+------+--------+------+-------+-----------------+----------------------+-------+---------------------------+----+------------------+-----+
|     2|       0|     0|    PRT|                0|                     0|   NULL|                          0|  BB|                 A|16119|
|     1|       0|     0|    PRT|                0|                     0|   NULL|                          0|  BB|                 A| 5462|
|     2|       0|     0|    FRA|                0|                     0|   NULL|                          0|  BB|                 A| 3781|
|     2|       0|     0|    PRT|                0|                     1|   NULL|


| least_children | most_children |
|----|----|
| 0 (110796 occurrences) | 10 (1 occurrences) |



| least_country | most_country |
|----|----|
| PRT (48590 occurrences) | GUY (1 occurrences) |



| least_company | most_company |
|----|----|
| NULL (112593 occurrences) | 54 (1 occurrences) |



| least_meal | most_meal |
|----|----|
| BB (92310 occurrences) | FB (798 occurrences) |



| least_reserved_room_type | most_reserved_room_type |
|----|----|
| A (85994 occurrences) | L (6 occurrences) |



 show number of unique categorical-values per column
+--------+-------+-------+----+------------------+
|children|country|company|meal|reserved_room_type|
+--------+-------+-------+----+------------------+
|       6|    178|    353|   5|                10|
+--------+-------+-------+----+------------------+


 print summary statistics of guest-numricals
+-------+------------------+--------------------+-------------------+----------------------+---------------------------+
|summary|            adults|              babies|  is_repeated_guest|previous_cancellations|required_car_parking_spaces|
+-------+------------------+--------------------+-------------------+----------------------+---------------------------+
|  count|            119390|              119390|             119390|                119390|                     119390|
|   mean|1.8564033838679956|0.007948739425412514|0.03191222045397437|   0.08711784906608594|        0.06251779881062065|
| stddev|0.5792609988327531|  0.0974361

# Business Question 1: what does our customer mix look like with regards to profitability?

profitability is going to be categorized by the colum "average-daily-rate" (adr) as follows:

- "no business"               -> adr_pp = (-infinity,0)
- "not profitable"            -> adr_pp = (0, 28)
- "break even"                -> adr_pp = (28,45) 
- "profitable"                -> adr_pp = (45,62) 
- "prioriy"                   -> adr_pp = (62,85)
- "VIP"                       -> adr_pp = (85,+infinity)



In [11]:
# 1. Let's calculate the avere-daily-rate per guest, (currently per booking).
# for this we need to create a total guests column, that takes into consideration, that children do not count as a "full guest" (also called PAX)
# then we devide the adr by PAX and categorize accordingly
# guest = guest + ["profitability"]

bq1_df = data_df.withColumn("PAX", (col("adults") + 0.5 * col("children") + 0.2 * col("babies")))\
                .withColumn("adr_pp", (col("adr") / col("PAX")))\
                    .withColumn("profitability", 
                     when(col("adr_pp")<=0,"no business")
                    .when((col("adr_pp")>0) & (col("adr_pp")<=28),"not profitable")
                    .when((col("adr_pp")>28) & (col("adr_pp")<=45),"break even")
                    .when((col("adr_pp")>45) & (col("adr_pp")<=62),"profitable")
                    .when((col("adr_pp")>62) & (col("adr_pp")<=85),"prioriy")
                    .otherwise("VIP"))

print('Print the customer mix according to the new categorization')
bq1_df.groupBy("profitability").count().sort(desc('count')).show(10)

Print the customer mix according to the new categorization
+--------------+-----+
| profitability|count|
+--------------+-----+
|    break even|37655|
|    profitable|32325|
|       prioriy|23224|
|           VIP|15347|
|not profitable| 9028|
|   no business| 1811|
+--------------+-----+



# Business Question 2: during which month do we get the highest ratio of profitable bookings

In [12]:
# we can reuse the df from business question 1 - however there are a few things to do:
# 1. cast dateTypes on the according columns, to create a date column
# 2. Calculate the booking date by subtracting the lead time from the 
# Group the data so it answers the Business Question

# first create a proper month column
bq2_df = bq1_df.withColumn("month",
            when(col("arrival_date_month") == "January", 1)
            .when(col("arrival_date_month") == "February", 2)
            .when(col("arrival_date_month")== "March", 3)
            .when(col("arrival_date_month") == "April", 4)
            .when(col("arrival_date_month")== "May", 5)
            .when(col("arrival_date_month")== "June", 6)
            .when(col("arrival_date_month")== "July", 7)
            .when(col("arrival_date_month")== "August", 8)
            .when(col("arrival_date_month") == "September", 9)
            .when(col("arrival_date_month") == "October", 10)
            .when(col("arrival_date_month") == "November", 11)
            .when(col("arrival_date_month") == "December", 12))

# next we can create a Date-colum "booking_date".
# finally, we can calculate the booking date, by subtracting the lead_time from the booking_date 
    # expr() is needed in order to pass the column to date_add    
bq2_df = bq2_df.withColumn("arrival_date", date_format(concat_ws('-', bq2_df.arrival_date_year, bq2_df.month , bq2_df.arrival_date_day_of_month), 'yyyy-MM-dd'))\
                .withColumn("booking_date", expr("date_add(to_date(arrival_date,'yyyy-MM-dd'),-cast(lead_time as int))"))\
                


# add dummy variables for profitability
bq2_df = bq2_df.withColumn("VIP", 
                    when(col("profitability") == "VIP", 1).otherwise(0))\
                    .withColumn("non_VIP", 
                    when(col("profitability") != "VIP", 1).otherwise(0))


# with the dataframe created above, we can finally go about answering the business question
# calculate ratio of "VIP" per month
ba2_df = bq2_df.groupBy(month("booking_date")).sum("VIP", "non_VIP")

ba2_df.withColumn(
    "ratio", round((col("sum(VIP)") / (col("sum(VIP)") + col("sum(non_VIP)"))),2)
).sort(desc("ratio")).show()


+-------------------+--------+------------+-----+
|month(booking_date)|sum(VIP)|sum(non_VIP)|ratio|
+-------------------+--------+------------+-----+
|                  6|    1562|        4501| 0.26|
|                  4|    1775|        6847| 0.21|
|                  5|    1641|        6212| 0.21|
|                  7|    1590|        8228| 0.16|
|                  8|    1243|        6727| 0.16|
|                  9|    1181|        7004| 0.14|
|                  3|    1332|        8589| 0.13|
|                 12|     836|        8332| 0.09|
|                  2|    1213|       12255| 0.09|
|                 11|     796|        8709| 0.08|
|                 10|     990|       11139| 0.08|
|                  1|    1188|       15500| 0.07|
+-------------------+--------+------------+-----+



# Business Question 3: what are the profitability ratios of each group by country
group by waitinglist, create profiling



- use pivot & join
- change prints to: display(Markdown("**Top 20 origin airports** with highest severe delayed (**unacceptable**) flights ratio (in \%):"))


"no business" -> adr_pp = (-infinity,0)
"not profitable" -> adr_pp = (0, 28)
"break even" -> adr_pp = (28,45)
"profitable" -> adr_pp = (45,62)
"prioriy" -> adr_pp = (62,85)
"VIP" -> adr_pp = (85,+infinity)

In [20]:
bq3_df = bq2_df.groupBy("country")\
                .pivot("profitability")\
                .agg(count("profitability"))\
                .orderBy(col("VIP").desc(), col("prioriy").desc(), col("profitable").desc(), col("break even").desc(), col("not profitable").desc())\
                .show()
                

+-------+----+----------+-----------+--------------+-------+----------+
|country| VIP|break even|no business|not profitable|prioriy|profitable|
+-------+----+----------+-----------+--------------+-------+----------+
|    PRT|6441|     17752|       1456|          3869|   8575|     10497|
|    ESP|1520|      2424|         72|           703|   1809|      2040|
|    FRA|1062|      3308|         42|           378|   2358|      3267|
|    GBR|1006|      3925|         68|          1877|   1952|      3301|
|    DEU| 803|      1945|         39|           327|   1338|      2835|
|    ITA| 452|       893|         12|           119|    787|      1503|
|    USA| 427|       424|          8|            75|    559|       604|
|    IRL| 293|      1079|          5|           521|    562|       915|
|    CHE| 292|       394|          5|            52|    429|       558|
|    NLD| 289|       501|          4|           159|    457|       694|
|    BEL| 252|       547|          8|           102|    552|    

In [ ]:
severeCrimeTimedDF.where((col("CrimeSeverity") == 9) | (col("CrimeSeverity") == 8) | (col("CrimeSeverity") == 7) | (col("CrimeSeverity") == 6))\
.na.drop()\
.groupBy("Intersection","DayOfWeek")\
.pivot("CrimeSeverity", ("9", "8", "7", "6"))\
.agg(count("CrimeSeverity").alias("NumOfOccurences"))\
.orderBy(col("9").desc(), col("8").desc(), col("7").desc(), col("6").desc())\
.show(20, False)